In [0]:
############################################################
# NETS 213: Homework 7
############################################################

import pandas as pd

# Part 1 - Majority vote

def majority_vote(mturk_res):
  data = mturk_res
  col1 = []
  col2 = []
  col3 = []
  l1 = []
  l2 = []
  l3 = []
  for i in range(0, len(data)):
    for j in range (1, 11):
      l1.append(data['Input.attr_id'][i])
      l2.append(data['Input.adj_' + str(j)][i])
      if data['Answer.adj_' + str(j)][i] == 'Yes':
        l3.append(1)
      else: 
        l3.append(0)
    for j in range(1, 6):
      l1.append(data['Input.attr_id'][i])
      l2.append(data['Input.pos_qual_ctrl_'+str(j)][i])
      if data['Answer.pos_qual_ctrl_'+str(j)][i] == 'Yes':
        l3.append(1)
      else:
        l3.append(0)
    l1.append(data['Input.attr_id'][i])
    l2.append(data['Input.neg_qual_ctrl'][i])
    if data['Answer.neg_qual_ctrl'][i] == "Yes":
      l3.append(1)
    else:
      l3.append(0)
  temp = pd.DataFrame(data={'attr':l1, 'adj':l2, 'bool':l3})
  agg = temp.groupby(['attr','adj']).mean().reset_index()
  for i in range(0,len(agg)):
    if agg['bool'][i] < 0.5:
      col1.append(agg['attr'][i])
      col2.append(agg['adj'][i])
      col3.append('No')
    else:
      col1.append(agg['attr'][i])
      col2.append(agg['adj'][i])
      col3.append('Yes')
  df = pd.DataFrame(data={'attr_id':col1,'adj':col2,'label':col3})
  return df

def majority_vote_workers(mturk_res, votes):
    data=mturk_res
#     print(data[data['WorkerId']=="A2OW2CIT4H7364"].shape)
    info=[]

    for index, row in data.iterrows():
        for i in range(1,11):
            info.append([row["WorkerId"],row['Input.attr_id'],row['Input.adj_'+str(i)],row["Answer.adj_"+str(i)]])
#         for i in range(1,6):
#             info.append([row["WorkerId"],row['Input.attr_id'],row['Input.pos_qual_ctrl_'+str(i)],row["Answer.pos_qual_ctrl_"+str(i)]])
#         info.append([row["WorkerId"],row['Input.attr_id'],row['Input.neg_qual_ctrl'],row["Answer.neg_qual_ctrl"]])

    df=pd.DataFrame(info,columns=['Workers','Attr','Adj','Answer'])
    print(df.head())
    workers=df['Workers'].unique()

    counts={}
    for w in workers:
        counts[w]=df[df["Workers"]==w].shape[0]

    labeldf = votes

    correct={}
    for index, row in df.iterrows():
        if (labeldf[(labeldf["attr_id"]==row["Attr"]) & (labeldf["adj"]==row["Adj"])]["label"]==row["Answer"]).iloc[0]:
            if row["Workers"] in correct.keys():
                correct[row["Workers"]]=correct[row["Workers"]]+1
            else:
                correct[row["Workers"]]=1
    
    print(correct)
    
    results=[[k,round(correct[k]/counts[k],3)] for k in counts.keys()]
    
    dfr=pd.DataFrame(results,columns=['worker_id', 'quality'])
    
    return dfr.sort_values(by="worker_id")

# Part 1 - Weighted majority vote

def weighted_majority_vote_workers(mturk_res):
    data=mturk_res
    info=[]
    
    count={}   
    correct={}
    for index, row in data.iterrows():
        c=0
        
        c=sum([row['Answer.neg_qual_ctrl']=='No',row['Answer.pos_qual_ctrl_1']=='Yes',row['Answer.pos_qual_ctrl_2']=='Yes',row['Answer.pos_qual_ctrl_3']=='Yes',row['Answer.pos_qual_ctrl_4']=='Yes',row['Answer.pos_qual_ctrl_5']=='Yes'])
        
        
        if row["WorkerId"] in correct.keys():
            correct[row["WorkerId"]]=correct[row["WorkerId"]]+c
        else:
            correct[row["WorkerId"]]=c
        
        if row["WorkerId"] in count.keys():
            count[row["WorkerId"]]=count[row["WorkerId"]]+6
        else:
            count[row["WorkerId"]]=6
    
#     print(correct)
    
    results=[[k,round(correct[k]/count[k],3)] for k in count.keys()]
    
    dfr=pd.DataFrame(results,columns=['worker_id', 'quality'])
#     print(dfr.head())
    
    return dfr.sort_values(by="worker_id")

def weighted_majority_vote(mturk_res, workers):
  data = mturk_res
  qual = workers
  col1 = []
  col2 = []
  col3 = []
  l1 = []
  l2 = []
  l3 = []
  for i in range(0, len(data)):
    mult = qual.loc[qual['worker_id'] == data['WorkerId'][i]].get_values()[0][2]
    for j in range (1, 11):
      l1.append(data['Input.attr_id'][i])
      l2.append(data['Input.adj_' + str(j)][i])
      if data['Answer.adj_' + str(j)][i] == 'Yes':
        l3.append(1 * mult)
      else: 
        l3.append(-1 * mult)
    for j in range(1, 6):
      l1.append(data['Input.attr_id'][i])
      l2.append(data['Input.pos_qual_ctrl_'+str(j)][i])
      if data['Answer.pos_qual_ctrl_'+str(j)][i] == 'Yes':
        l3.append(1 * mult)
      else:
        l3.append(-1 * mult)
    l1.append(data['Input.attr_id'][i])
    l2.append(data['Input.neg_qual_ctrl'][i])
    if data['Answer.neg_qual_ctrl'][i] == "Yes":
      l3.append(1 * mult)
    else:
      l3.append(-1 * mult)
  temp = pd.DataFrame(data={'attr':l1, 'adj':l2, 'bool':l3})
  agg = temp.groupby(['attr','adj']).mean().reset_index()
  for i in range(0,len(agg)):
    if agg['bool'][i] < 0:
      col1.append(agg['attr'][i])
      col2.append(agg['adj'][i])
      col3.append('No')
    else:
      col1.append(agg['attr'][i])
      col2.append(agg['adj'][i])
      col3.append('Yes')
  df = pd.DataFrame(data={'attr_id':col1,'adj':col2,'label':col3})
  return df


# Part 2 - EM algorithm

def em_worker_quality(rows, labels):
    pass

def em_votes(rows, worker_qual):
    pass

def em_iteration(rows, worker_qual):
    labels = em_votes(rows, worker_qual)
    worker_qual = em_worker_quality(rows, labels)
    return labels, worker_qual

def em_vote(rows, iter_num):
    urls={}
    conf=[]
    for inx, row in rows.iterrows():
        if row.iloc[1] in urls.keys():
            urls[row.iloc[1]].append((row.iloc[0],row.iloc[2]))
        else:
            urls[row.iloc[1]]=[(row.iloc[0],row.iloc[2])]
    conf=[]
    for k in urls.keys():
        temp=[]
        temp.append(k)
        for v in urls[k]:
            temp.append(v[1])
        conf.append(temp)
    df=pd.DataFrame(conf,columns=['urls','worker1','worker2','worker3','worker4','worker5'])

    
    #create initial label dataframe to reflect the initial accuracies of the workers
    labels=[]
    pretend=[]
    for inx,row in df.iterrows():
        count=sum([row['worker1']=='porn',row['worker2']=='porn',row['worker3']=='porn',row['worker4']=='porn',row['worker5']=='porn'])
        labels.append([count/5,1-(count/5)])
        if count/5 <0.5:
            pretend.append([0,1])
        else:
            pretend.append([1,0])
    
    labels=pd.DataFrame(labels,columns=['porn','not porn'])
    pretend=pd.DataFrame(pretend,columns=['porn','not porn'])
    
    for i in range(iter_num):
    
        #create workerqualities in the form of truth tables for each worker using the labels
        wq=[]
        for column in [df['worker1'],df['worker2'],df['worker3'],df['worker4'],df['worker5']]:
            tt=[0,0,0,0]
            for el in range(len(column)):
                
                #get the "true" label for this website
                if pretend.iloc[el,0]==1:
                    truth='porn'
                else:
                    truth='notporn'
                    
                #check what the worker said about this website and update the truth table
                if column[el]=='porn':
                    if truth=='porn':
                        tt[0]=tt[0]+1
                    else:
                        tt[2]=tt[2]+1
                if column[el]=='notporn': 
                    if truth=='porn':
                        tt[1]=tt[1]+1
                    else:
                        tt[3]=tt[3]+1

            for i in range(len(tt)):
                tt[i]=tt[i]/5
            wq.append(tt)

        wq=pd.DataFrame(wq)
        
        print("Labels with percentages"+str(labels))

        #create brand new estimates for labels using worker feedback and their truth table values
        labels[:] = 0
        for inx,row in df.iterrows():

            for i in range(1,6):
                guess=row['worker'+str(i)]

                if guess=='porn':
                    labels["porn"].iloc[inx]+=wq.iloc[i-1,0]
                    labels["not porn"].iloc[inx]+=wq.iloc[i-1,2]
                else:
                    labels["porn"].iloc[inx]+=wq.iloc[i-1,1]
                    labels["not porn"].iloc[inx]+=wq.iloc[i-1,3]
        
        #convert to percentages
        #create the binary version of the labels for the next iteration
        for inx,row in labels.iterrows():
            summ=row["porn"]+row["not porn"]
            labels["porn"].iloc[inx]= labels["porn"].iloc[inx]/summ
            labels["not porn"].iloc[inx]= labels["not porn"].iloc[inx]/summ
            
            if labels["porn"].iloc[inx]<0.5:
                pretend["porn"].iloc[inx]=0
                pretend["not porn"].iloc[inx]=1
            else:
                pretend["porn"].iloc[inx]=1
                pretend["not porn"].iloc[inx]=0
    
    result=[]
    for i,row in pretend.iterrows():
        result.append([df["urls"].iloc[i],'porn' if pretend["porn"].iloc[i]==1 else "notporn"])
    result=pd.DataFrame(result,columns=['url','label'])
    return result


# Part 3 - Qualified workers

def select_qualified_worker(mturk_res):
  data = mturk_res
  l1 = []
  l2 = []
  
  for i in range(0, len(data)):
    poscorrect = 0
    negcorrect = True
    l1.append(data['WorkerId'][i])
    for j in range(1,6):
      if data['Answer.pos_qual_ctrl_'+str(j)][i] == 'Yes':
        poscorrect += 1
    if data['Answer.neg_qual_ctrl'][i] == 'Yes':
      negcorrect = False
    if poscorrect >= 4 and negcorrect:
      l2.append(1)
    else:
      l2.append(0)
  temp = pd.DataFrame(data={'worker_id':l1, 'percentage':l2})
  agg = temp.groupby(['worker_id']).mean().reset_index()
  '''
  col1 = []
  col2 = []
  for i in range (0, len(agg)):
    print(agg['percentage'][i])
    if agg['percentage'][i] >= 0.8:
      col1.append(agg['worker_id'][i])
      col2.append(agg['percentage'][i])
  '''
  
  return agg


# Your main function

def main():
    # Read in CVS result file with pandas
    # PLEASE DO NOT CHANGE
    mturk_res = pd.read_csv('hw7_data.csv')

    # Call functions and output required CSV files
    df = majority_vote(mturk_res)
    df.to_csv('output1.csv')
    df2=majority_vote_workers(mturk_res, df)
    df2.to_csv('output2.csv')
    df3=weighted_majority_vote_workers(mturk_res)
    df3.to_csv('output3.csv')
    worker = pd.read_csv('output3.csv')
    weighted = weighted_majority_vote(mturk_res, worker)
    weighted.to_csv('output4.csv')
    rows = pd.read_csv('em_toy_data.txt', sep="\t", header=None)
    df4=em_vote(rows,3)
    df4.to_csv('output5.csv')
    sqw = select_qualified_worker(mturk_res)
    sqw.to_csv('output6.csv')
    
    pass

if __name__ == '__main__':
    main()

          Workers         Attr           Adj Answer
0  A1MIBEGF81ZLUS  beauty.n.01  inconclusive     No
1  A1MIBEGF81ZLUS  beauty.n.01        bright    Yes
2  A1MIBEGF81ZLUS  beauty.n.01      inclined     No
3  A1MIBEGF81ZLUS  beauty.n.01      succinct     No
4  A1MIBEGF81ZLUS  beauty.n.01       serious     No
{'A1MIBEGF81ZLUS': 55, 'A1X1IRFYQZ40LD': 10, 'A1ZGQC0J00M69W': 1615, 'A2OW2CIT4H7364': 1301, 'A1AR5OZLXIF9': 47, 'A2UF2FRGVW4T89': 1187, 'A1RUURPQJ14A8X': 1055, 'A31A4YKVSOYRVS': 260, 'AA21VG9GOHM54': 62, 'A2M5VW97GIYLHB': 424, 'AOBTFLJU2BEPR': 703, 'A3LN9BVLOWLYER': 299, 'ARYYSG1HT2UDX': 3169, 'AJQGWGESKQT4Y': 88, 'A3GS6QRW0NDZV9': 213, 'A1PC5HFTBIR2QA': 7, 'A3KA3A3M7LPMPA': 390, 'A3S8TUZ94SKSDC': 53, 'A2HR7ZIX42FEPG': 713, 'A37CE37LHUN8M6': 722, 'A3GUJ6JD25FX7O': 623, 'A2S09R2TB55WBN': 81, 'A2HTN7HCNBQ8V4': 347, 'A15XX0WDRG29E7': 301, 'A3542PB4Z0NE80': 321, 'A3RS7UCO7CQ74R': 1520, 'A3P57IUDHUKNCE': 838, 'A1EBVJWBNNPUFU': 107, 'A1BW8BW8LB6ZPC': 59, 'A2RCYLKY072XXO': 17, 'A19TL7A